заменить: Служб Провайдер обслуж услуг клиент
термины: поставщик потребитель, развитие сервиса, сообщество сервиса

<h1>БРОСКИЙ ЗАГОЛОВОК или "Контракты для потребителей" как путь развития сервиса (перевод)</h1>
<blockquote>
КАРТИНКА ДЛЯ ПРИВЛЕЧЕНИЯ ВНИМАНИЯ

МОТИВИРУЮЩЕЕ ВСТУПЛЕНИЕ
Зачем я перевел старьё 2006 года? По двум причинам:
1. На нее ссылается Мартин Фаулер в своей <a href="https://martinfowler.com/articles/microservices.html">основополагающей статье</a> про микросервисный архитектурный стиль. Он пишет:<br>
Шаблоны, такие как Tolerant Reader и Consumer-Driven Contracts, часто применяются к микросервисам. Они помогают развивать контракты сервисов независимо. Проверка контрактов с потребителями в вашем сборочном конвейере повышает уверенность и обеспечивает быструю обратную связь: работает ли ваш сервис? Действительно, мы знаем о команде в Австралии, которая управляет созданием новых сервисов через "контракты потребителей". Они используют простые инструменты, которые позволяют им определять контракт на сервис, включенные в сборочный конвейер сервиса. Сервис собирается только тогда, когда он удовлетворяет контракту - изящный подход, чтобы избежать дилеммы «YAGNI» [9]. Эти методы и инструменты вокруг них, ограничивают потребность в централизованном управлении контрактами, уменьшая релизные зависимости между сервисами.
2. В русскоязычных источниках информации по этому вопросу - раз-два и обчёлся.
ССЫЛКИ НА ИСТОЧНИКИ НА РУССКОМ
<a href="https://habr.com/post/358950/">Пирамида тестов на практике</a>, раздел "Контрактные тесты".
<a href="https://habr.com/company/avito/blog/333644/">Ядро автоматизации тестирования в микросервисной архитектуре</a>

Для кого эта статья? Для всех, кто разрабатывает собственные сервисы или потребляет (как разработчик) чужие сервисы. 
Оригинальная статья за авторством <a href="http://iansrobinson.com/">Яна Робинсона</a> опубликована на сайте Фаулера: <a href="https://martinfowler.com/articles/consumerDrivenContracts.html">https://martinfowler.com/articles/consumerDrivenContracts.html</a>
Я дополнил оригинальный текст мотивирующими зарисовками из практики,а также убрал неактуальный раздел, описывающий расширение схем XML. Дополнения оформлены врезками (как это вступление).
</blockquote>
<cut />
<i>В этой статье обсуждаются проблемы, возникающие между разработчиками и потребителями сервисов, например, когда поставщики меняют часть своего контракта, в частности, схемы документов. Описываются две стратегии борьбы с ними:
    1. Добавление точек расширения.
    2. "Достаточная" валидация получаемых сообщений. 
Обе стратегии помогают защитить потребителей от изменений в контракте, но не дают провайдеру сервиса понимания:
- Как их лучше использовать.
- Что необходимо делать по мере развития сервиса.

В статье описан шаблон <b>Контракты для потребителей</b>,  который позволяет поставщикам лучше понимать своих потребителей и фокусирует развитие сервиса на том, что нужно потребителям.</i>
<source lang="xml">

<h2>Пример эвелюции сервиса</h2>
Чтобы проиллюстрировать некоторые проблемы, с которыми мы сталкиваемся при разработке сервисов, рассмотрим простой сервис ProductSearch, который позволяет потребительским приложениям осуществлять поиск в нашем каталоге продуктов. Результат поиска имеет следующую структуру:<img src ='cdc\products1.gif'>
<center>_Рисунок 1: Схема результата поиска_</center>

Примерный документ с результатами поиска выглядит следующим образом:
```xml
<?xml version="1.0" encoding="utf-8"?>
<Products xmlns="urn:example.com:productsearch:products">
  <Product>
    <CatalogueID>101</CatalogueID>
    <Name>Widget</Name>
    <Price>10.99</Price>
    <Manufacturer>Company A</Manufacturer>
    <InStock>Yes</InStock>
  </Product>
  <Product>
    <CatalogueID>300</CatalogueID>
    <Name>Fooble</Name>
    <Price>2.00</Price>
    <Manufacturer>Company B</Manufacturer>
    <InStock>No</InStock>
  </Product>
</Products>
```
Сервис ProductSearch в настоящее время используется двумя приложениями: внутренним маркетинговым приложением и веб-приложением внешнего реселлера. Оба клиента используют проверку XSD для проверки принятых документов до их обработки. Внутреннее приложение использует поля CatalogueID, Name, Price и Manufacturer; внешнее приложение - поля CatalogueID, Name и Price. Ни одно из них не использует поле InStock: хотя оно рассматривается для маркетингового приложения, оно было выкинуто на раннем этапе жизненного цикла разработки.

Одним из наиболее распространенных способов развития сервиса является добавление поля в документ для одного или нескольких 
потребителей. В зависимости от того, как клиентская и серверная части были реализованы, даже простые изменения, подобные этому, могут иметь дорогостоящие последствия для бизнеса и его партнеров.

В нашем примере, после того, как сервис ProductSearch была в продакшн в течение некоторого времени, второй реселлер рассматривает его использование, но просит, чтобы поле <i>Description</i> будет добавлено к каждому продукту. Из-за того, как сделаны клиенты, это изменение имеет значительные и дорогостоящие последствия как для провайдера, так и для существующих потребителей. Стоимость каждого из них зависит от того, как мы реализуем изменения. Есть как минимум два способа, которыми мы можем распределять стоимость изменений между участниками "сообщества сервиса". Во-первых, мы могли бы изменить нашу исходную схему и потребовать, чтобы каждый потребитель обновил свою копию схемы, чтобы правильно проверить результаты поиска; стоимость изменения системы здесь распределяется между провайдером - который, столкнувшись с таким запросом на изменение, всегда будет делать какие-то изменения - и потребителями, которые не интересуются обновленной функциональностью. С другой стороны, мы могли бы добавить вторую операцию и схему для поставщика услуг для нового потребителя и сохранить исходную операцию и схему для существующих потребителей. Все доработки теперь только на стороне провайдера, но за счет повышения сложности и стоимости поддержки сервиса.

<h2>Интерлюдия: утомленные сервисами</h2>
Главными преимуществами использования сервисов в корпоративных системах являются:
1. Повышение организационной гибкости
2. Cнижение общих затрат на реализацию изменений.

SOA повышает гибкость за счет размещения бизнес-функций в выделенных, переиспользуемых сервисах. Затем эти сервисы используются и оркестрируются для выполнения основных бизнес-процессов. Это снижает стоимость изменений за счет сокращения зависимостей между сервисами, позволяя им быстро перестраиваться и настраиваться в ответ на изменения или незапланированные события.

Однако бизнес может полностью реализовать эти преимущества, только если его SOA позволяет сервисам эволюционировать независимо друг от друга. Чтобы повысить независимость, мы создаем контракты сервисов. Несмотря на это, мы вынуждены дорабатывать потребителей так же часто, как и поставщика, главным образом потому, что потребители зависеть от конкретной версии контракта поставщика. В конце концов, поставщики начинают крайне осторожно подходить к изменению любого элемента контракта; это, в частности, потому, что они не имеют представления о том, как потребители реализуют этот контракт. В худшем случае потребители завязываются на поставщика, наивно описывая всю схему документа в рамках своей внутренней логики.

Контракты обеспечивают независимость сервисов; парадоксально, что они могут также связывать поставщиков и потребителей нежелательными способами. Не задумываясь о функции и роли контрактов, которые мы реализуем в нашей SOA, мы подвергаем наши сервисы  «скрытой» связи. Отсутствие какой-либо информации о том, как потребители сервиса реализуют контракт в коде, а также отсутствие ограничений на реализацию (и для поставщика, и для потребителя), в совокупности подрывают предполагаемые преимущества SOA, позволяющие предприятию. Короче говоря, предприятие начинает утомляться от сервисов.

## Версионирование схемы ##
Мы можем начать исследование проблем контракта и зависимостей, мешающих нашему сервису ProductSearch, с версионирования схемы. Группа технической архитектуры WC3 (TAG) описала ряд стратегий управления версиями, которые могут помочь нам разработать схемы сообщений для наших сервисов способами, которые уменьшают наши проблемы с зависимостью. Эти стратегии варьируются от чрезмерно либеральной _none_, которая требует, чтобы сервисы не различали разные версии схемы и принимали все изменения, до чрезвычайно консервативного _большого взрыва_, что требует от сервиса выдавать ошибку, если он получает неожиданную версию сообщения.

Обе крайности приносят с собой проблемы, которые не мещают получить пользу для бизнес и увеличивают общую стоимость владения системой. Явные и неявные стратегии «без версий» приводят к системы, которые непредсказуемы в своих взаимодействиях, плохо развиваемы и имеют высокую стоимость доработок. С другой стороны, стратегии "большого взрыва" создают сильно связанные сервисные ландшафты, в которых изменения схемы затрагивают всех поставщиков и потребителей, ухудшая доступность, замедляя развитие и уменьшая возможности для получения прибыли.

"Сообщество сервиса" из нашего примера эффективно реализует стратегию большого взрыва. Учитывая затраты, связанные с повышением бизнес-ценности системы, ясно, что поставщики и потребители выиграют от более гибкой стратегии управления версиями - той, что TAG называет стратегией _совместимости_. Она обеспечивает прямую и обратную совместимость схем. При развитии сервисов схемы с обратной совместимостью позволяют потребителям новых схем принимать экземпляры старой схемы: поставщик, созданный для обработки обратно совместимых новых версий, может, тем не менее, принять запрос в формате старой схемы. С другой стороны, прямая совместимость позволяет потребителям старых схем обрабатывать экземпляр новой схемы. Это ключевой момент для существующих пользователей ProductSearch: если бы схема результатов поиска изначально была бы сделана с учетом прямой совместимости, потребители могли бы обрабатывать ответы новой версии, не нуждаясь в доработке.

### Точки расширения ###
Составление схем c обратной и прямой совместимостью - это хорошо понятная задача дизайна, лучше всего выраженная шаблоном расширяемости _Must Ignore_ (см. статьи <a href="http://www.pacificspirit.com/Authoring/Compatibility/ExtendingAndVersioningXMLLanguages.html">Дэвида Орчарда</a> и <a href="http://msdn.microsoft.com/library/en-us/dnexxml/html/xml07212004.asp">Дейр Обасанжо</a>). Шаблон _Must Ignore_ рекомендует, чтобы схемы включали точки расширяемости, которые позволяют добавлять элементы к типам и дополнительные атрибуты для каждого элемента. В шаблоне также рекомендуется, чтобы XML описывал модель обработки, которая определяет, что потребителям делать с расширениями. Простейшая модель требует от потребителей игнорировать элементы, которые они не распознают - отсюда и название шаблона. Модель может также потребовать от потребителей обработки элементов, имеющих флаг «Must Understanding», либо выдать ошибку, если они не могут их понять.

Вот наша изначальная схема документа с результатами поиска:
```xml
<?xml version="1.0" encoding="utf-8"?>
<xs:schema xmlns="urn:example.com:productsearch:products" 
  xmlns:xs="http://www.w3.org/2001/XMLSchema"
  elementFormDefault="qualified" 
  targetNamespace="urn:example.com:productsearch:products" 
  id="Products">
  <xs:element name="Products" type="Products" />
  <xs:complexType name="Products">
    <xs:sequence>
      <xs:element minOccurs="0" maxOccurs="unbounded" name="Product" type="Product" />
    </xs:sequence>
  </xs:complexType>
  <xs:complexType name="Product">
    <xs:sequence>
      <xs:element name="CatalogueID" type="xs:int" />
      <xs:element name="Name" type="xs:string" />
      <xs:element name="Price" type="xs:double" />
      <xs:element name="Manufacturer" type="xs:string" />
      <xs:element name="InStock" type="xs:string" />
    </xs:sequence>
  </xs:complexType>
</xs:schema>
```
Давайте теперь откатимся во времени и с самого начала укажем для нашего сервиса совместимую, расширяемую схему:
```xml
<?xml version="1.0" encoding="utf-8"?>
<xs:schema xmlns="urn:example.com:productsearch:products" 
  xmlns:xs="http://www.w3.org/2001/XMLSchema"
  elementFormDefault="qualified" 
  targetNamespace="urn:example.com:productsearch:products" 
  id="Products">
  <xs:element name="Products" type="Products" />
  <xs:complexType name="Products">
    <xs:sequence>
      <xs:element minOccurs="0" maxOccurs="unbounded" name="Product" type="Product" />
    </xs:sequence>
  </xs:complexType>
  <xs:complexType name="Product">
    <xs:sequence>
      <xs:element name="CatalogueID" type="xs:int" />
      <xs:element name="Name" type="xs:string" />
      <xs:element name="Price" type="xs:double" />
      <xs:element name="Manufacturer" type="xs:string" />
      <xs:element name="InStock" type="xs:string" />
      <xs:element minOccurs="0" maxOccurs="1" name="Extension" type="Extension" />
    </xs:sequence>
  </xs:complexType>
  <xs:complexType name="Extension">
    <xs:sequence>
      <xs:any minOccurs="1" maxOccurs="unbounded" namespace="##targetNamespace" processContents="lax" />
    </xs:sequence>
  </xs:complexType>
</xs:schema>
```
Эта схема включает дополнительный элемент расширения для каждого продукта. Сам элемент расширения может содержать один или несколько элементов из целевого пространства имен:<img src="cdc\products2.gif">
<center>_Рисунок 2: Расширяемая схема результата поиска_</center>

Теперь, когда мы получаем запрос на добавление описания к каждому продукту, мы можем опубликовать новую схему с дополнительным элементом _Description_, который поставщик вставляет в точку расширения. Это позволяет сервису ProductSearch возвращать результаты, содержащие описания продуктов, и потребители, использующие новую схему, для проверки всего документа. Потребители, использующие старую схему, не сломаются, хотя они не будут обрабатывать описание. Новый документ с результатом выглядит следующим образом:
```xml
<?xml version="1.0" encoding="utf-8"?>
<Products xmlns="urn:example.com:productsearch:products">
  <Product>
    <CatalogueID>101</CatalogueID>
    <Name>Widget</Name>
    <Price>10.99</Price>
    <Manufacturer>Company A</Manufacturer>
    <InStock>Yes</InStock>
    <Extension>
      <Description>Our top of the range widget</Description>
    </Extension>
  </Product>
  <Product>
    <CatalogueID>300</CatalogueID>
    <Name>Fooble</Name>
    <Price>2.00</Price>
    <Manufacturer>Company B</Manufacturer>
    <InStock>No</InStock>
    <Extension>
      <Description>Our bargain fooble</Description>
    </Extension>
  </Product>
</Products>
```
Пересмотренная схема выглядит так:
```xml
<?xml version="1.0" encoding="utf-8"?>
<xs:schema xmlns="urn:example.com:productsearch:products" 
  xmlns:xs="http://www.w3.org/2001/XMLSchema"
  elementFormDefault="qualified" 
  targetNamespace="urn:example.com:productsearch:products" 
  id="Products">
  <xs:element name="Products" type="Products" />
  <xs:complexType name="Products">
    <xs:sequence>
      <xs:element minOccurs="0" maxOccurs="unbounded" name="Product" type="Product" />
    </xs:sequence>
  </xs:complexType>
  <xs:complexType name="Product">
    <xs:sequence>
      <xs:element name="CatalogueID" type="xs:int" />
      <xs:element name="Name" type="xs:string" />
      <xs:element name="Price" type="xs:double" />
      <xs:element name="Manufacturer" type="xs:string" />
      <xs:element name="InStock" type="xs:string" />
      <xs:element minOccurs="0" maxOccurs="1" name="Extension" type="Extension" />
    </xs:sequence>
  </xs:complexType>
  <xs:complexType name="Extension">
    <xs:sequence>
      <xs:any minOccurs="1" maxOccurs="unbounded" namespace="##targetNamespace" processContents="lax" />
    </xs:sequence>
  </xs:complexType>
  <xs:element name="Description" type="xs:string" />
</xs:schema>
```
Обратите внимание, что первая версия расширяемой схемы совместима со второй, а вторая - обратно совместима с первой. Однако эта гибкость происходит за счет повышения сложности. Расширяемые схемы позволяют нам делать непредвиденные изменения, но они дают возможности, которые могут быть никогда не востребованы; при этом мы теряем выразительность, исходящую от простого дизайна и  понятное представление данных, вводя метаинформационные элементы контейнера.
Здесь мы не будем обсуждать расширяемость схемы. Достаточно сказать, что точки расширения позволяют нам делать прямо и обратно совместимые изменения в схемах и документах, не затрагивая поставщиков и потребителей сервиса. Однако расширение схемы не помогает нам управлять эволюцией системы, когда нам нужно сделать то, что якобы является нарушением контракта.

## Несовместимые изменения ##
Наш сервис ProductSearch включает в результаты поиска поле, указывающее, есть ли в наличии данный товар. Сервис заполняет это поле, используя дорогостоящий вызов в древнюю систему инвентаризации - зависимость, которую дорого поддерживать. Поставщик хотел бы удалить эту зависимость, очистить дизайн и улучшить общую производительность системы - желательно, без влияния на потребителей. Общаясь с потребителями, команда поставщика выясняет, что ни одно из потребительских приложений на самом деле ничего не делает с этим полем; хотя и дорогостоящее, оно избыточно.

К сожалению, в текущем положении, если мы удалим поле InStock из нашей расширяемой схемы, мы сломаем существующих потребителей. Чтобы исправить поставщика, мы должны исправить всю систему: когда мы удаляем функциональность от поставщика и публикуем новый контракт, каждое приложение-потребитель нужно будет переустановить с новой схемой, а взаимодействие между сервисами тщательно протестировать. Сервис ProductSearch в этом отношении не может развиваться независимо от потребителей: поставщик и потребители должны все "прыгать одновременно".

Наше сообщество сервиса разочаровывается в его эволюции, потому что каждый потребитель имеет «скрытую» зависимость, которая отражает весь контракт поставщика во внутренней логике потребителя. Потребители, используя валидацию по XSD и, в меньшей степени, статическую привязку к схеме документа в коде, неявно принимают весь контракт поставщика, независимо от их намерений использовать лишь часть.

Дэвид Орчард дает некоторые подсказки относительно того, как мы могли бы избежать этой проблемы, когда ссылается на <a href="https://en.wikipedia.org/wiki/Robustness_principle">принцип надежности</a>: «Реализация должна быть консервативной в своем поведении и либеральной в том, что принимает от других». Мы можем усилить этот принцип в контексте эволюции обслуживания, заявив, что приемники сообщений должны выполнять «достаточную» проверку: то есть они должны обрабатывать только используемые ими данные, и должны выполнять явно ограниченную или целенаправленную проверку полученных ими данных - в отличие от неявно неограниченной валидации «все или ничего», присущей обработке XSD.

### Schematron ###
Один из способов, которым мы можем настроить проверку на стороне покупателя - это использование масок или регулярных выражений для путей до элементов документа, возможно, с использованием языка валидации структуры дерева, такого как <a href="http://www.schematron.com/">Schematron</a>. Используя Schematron, каждый пользователь сервиса ProductSearch может задать, что он ожидает найти в результатах поиска:
```xml
<?xml version="1.0" encoding="utf-8" ?>
<schema xmlns="http://www.ascc.net/xml/schematron">

  <title>ProductSearch</title>
  <ns uri="urn:example.com:productsearch:products" prefix="p"/>
  
  <pattern name="Validate search results">
    <rule context="*//p:Product">
      <assert test="p:CatalogueID">Must contain CatalogueID node</assert>
      <assert test="p:Name">Must contain Name node</assert>
      <assert test="p:Price">Must contain Price node</assert>
    </rule>
  </pattern>
```
Реализации Schematron обычно преобразуют схему Schematron, такую как эта, в XSLT-преобразование, которое получатель сообщения может применить к документу для проверки на корректность.

Обратите внимание, что эта схема Schematron не содержит предположений об элементах в базовом документе, которые не нужны приложению-потребителю. Таким образом, явно описана валидация только требуемых элементов. Изменения схемы базового документа не будут отвергнуты при валидации, если они не нарушают явные ожидания, описанные в схеме Schematron. Даже если это удаление ранее обязательных элементов.

Вот относительно легкое решение наших проблем с контрактом и зависимостью, и это не требует от нас добавления неявных метаинформационных элементов в документ. Итак, давайте снова откатимся назад во времени и восстановим простую схему, описанную в начале статьи. Но на этот раз мы также будем настаивать на том, что потребители свободны в своем поведении и валидируют и обрабатывают только ту информацию, которая им необходима (используя схемы Schematron, а не XSD для проверки принятых сообщений). Теперь, когда поставщику предлагается добавить описание для каждого продукта, сервис может опубликовать пересмотренную схему, не затрагивая существующих потребителей. Аналогичным образом, обнаружив, что поле InStock не проверяется или не обрабатывается ни одним из потребителей, сервис может пересмотреть схему результатов поиска - снова, не затрагивая потребителей.

В конце этого процесса схема ответа ProductSearch выглядит так:
```xml
<?xml version="1.0" encoding="utf-8"?>
<xs:schema xmlns="urn:example.com:productsearch:products" 
  xmlns:xs="http://www.w3.org/2001/XMLSchema"
  elementFormDefault="qualified" 
  targetNamespace="urn:example.com:productsearch:products" 
  id="Products">
  <xs:element name="Products" type="Products" />
  <xs:complexType name="Products">
    <xs:sequence>
      <xs:element minOccurs="0" maxOccurs="unbounded" name="Product" type="Product" />
    </xs:sequence>
  </xs:complexType>
  <xs:complexType name="Product">
    <xs:sequence>
      <xs:element name="CatalogueID" type="xs:int" />
      <xs:element name="Name" type="xs:string" />
      <xs:element name="Price" type="xs:double" />
      <xs:element name="Manufacturer" type="xs:string" />
      <xs:element name="Description" type="xs:string" />
    </xs:sequence>
  </xs:complexType>
</xs:schema>
```

## Контракты для потребителей ###
Использование Schematron в приведенном выше примере приводит к некоторым интересным выводам о контрактах между поставщиками и потребителями, выходящим за рамки проверки документов. В этом разделе мы выделяем и обобщаем некоторые из этих идей и выражаем их в терминах шаблона, который мы называем _Контракты потребителей_.

Первое, что нужно отметить, это то, что схемы документов являются лишь частью того, что поставщик сервиса должен предлагать потребителям, чтобы они могли использовать его функциональность. Мы называем эту предназначенную для внешних потребителей информацию _контрактом поставщика_.

### Контракты поставщиков ###
Контракт поставщика описывает функциональность сервиса в виде набора экспортируемых элементов, необходимых для использования этой функциональности. С точки зрения развития сервиса контракт представляет собой контейнер для экспортируемого набора элементов бизнес-функций. Список этих элементов включает:
* __Схемы документов__. Мы уже подробно обсудили схемы документов. Наряду с интерфейсами, схемы документов являются частями контракта поставщика, изменение которых наиболее вероятно по мере развития сервиса; но, возможно, из-за этого, они также являются частями, с которыми мы имеем больше всего опыта внедрения со стратегиями развития сервисов, такими, как точки расширения и использование масок для путей к элементам документа.
* __Интерфейсы__. В простейшей форме интерфейсы поставщика включают набор экспортируемых сигнатур операций, которые потребитель может использовать для управления поведением поставщика. Системы обмена сообщениями обычно экспортируют относительно простые сигнатуры операций и помещают бизнес-содержимое внутрь сообщений, которые они обменивают. В таких системах принятые сообщения обрабатываются в соответствии с семантикой, закодированной в заголовке сообщения или полезной нагрузке. RPC-подобные сервисы, с другой стороны, кодируют большую часть своей бизнес-семантики в сигнатурах своих операций. В любом случае, потребители зависят от некоторой части интерфейса поставщика, и, следовательно, мы должны учитывать  интерфейс при развитии нашего сервиса.
* __Диалоги__. Поставщики и потребители обмениваются сообщениями в последовательностях, которые составляют один или несколько шаблонов обмена сообщениями, таких как "запрос-ответ" или "fire-and-forget". В ходе диалога потребитель может ожидать, что провайдер будет находиться в некотором конкретном контексте при отправке и получении сообщений. Например, сервис бронирования отелей может предложить потребителям возможность зарезервировать номер в начале диалога, а также подтвердить бронирование и внести депозит в последующих обменах сообщениями. Потребитель здесь может разумно ожидать, что сервис «запомнит» детали резервирования при проведении этих последующих обменов, вместо того, чтобы требовать, чтобы стороны повторяли весь разговор на каждом этапе процесса. По мере развития сервиса, набор "диалоговых гамбитов", доступных поставщику и потребителю, может измениться. Таким образом, диалоги являются кандидатами на то, чтобы считаться частью контракта поставщика.
* __Политики__. Помимо экспорта схем документов, интерфейсов и диалогов, поставщики могут объявлять и применять конкретные требования к использованию, которые определяют, как можно реализовать другие элементы контракта. Чаще всего эти требования относятся к контекстам безопасности и транзакций, в которых потребитель может использовать функциональность сервиса. Web-сервисы могут выражать это с использованием общей модели WS-Policy и дополнительных спецификаций, таких как WS-SecurityPolicy, но в контексте рассматриваемых нами "политик как кандидатов для включения в контракт поставщика", наше определение политики не привязано к конкретной спецификации или реализации.
* __Характеристики качества обслуживания__. Бизнес-ценность, которую дают поставщики и потребители сервиса, часто оценивается в контексте специфических характеристик качества обслуживания, таких как доступность, латентность и пропускная способность. Мы должны учитывать эти характеристики как вероятные составляющие контракта поставщика и учитывать их в наших стратегиях развития сервиса.

Это определение контракта немного шире, чем то, которое мы обычно подразумеваем, говоря о сервисах, но, с точки зрения развития сервиса оно полезно. Мы не должны считать его исчерпывающим в отношении типов элементов, которые может содержать контракт поставщика: это просто логичный набор элементов, которые являются кандидатами на включение в стратегию развития сервиса. С логической точки зрения этот набор элементов-кандидатов открыт, но на практике внутренние или внешние факторы, такие как требования к совместимости или ограничения платформы, могут ограничивать тип элементов, которые может содержать контракт. Например, контракт сервиса, который соответствует профилю WS-Basic, не будет содержать политик.

Невзирая на любые подобные ограничения, область контракта определяется просто связностью его элементов. Контракт может содержать много элементов и быть широким по охвату, или сосредоточиться лишь на немногих, до тех пор, пока этого достаточно для описания его функциональности.

Как мы решаем, включать ли элемент в контракт поставщика? Мы делаем это, спрашивая себя, может ли любой из наших потребителей сформулировать (например, в виде тестов) одно или несколько ожиданий в отношении функциональности, описываемой элементом. Мы уже видели из нашего примера сервиса, как потребители  могут проявлять интерес к частям схемы документа, экспортируемой сервисом, и как они могут с помощью тестов утверждать, что их ожидания относительно этого контрактного элемента по-прежнему выполняются. Поэтому наша схема документа является частью нашего контракта поставщика.

Контракты поставщиков имеют следующие характеристики:
* __Закрытые и полные__. Контракты поставщика выражают функциональность сервиса через полный набор доступных для потребителей экспортируемых элементов, и, как таковые, являются закрытыми и полными в отношении доступных для использования функциональных возможностей.
* __Единственные и обязательные__. Контракты поставщика являются единственным и обязывающим описанием функциональности сервиса.
* __Ограниченная стабильность и неизменность__. Контракт поставщика стабилен и неизменен в течение ограниченного периода и/или области действия (см. раздел  "Validity of Data in Bounded Space and Time" в статье Пэта Хелланда <a href="http://cidrdb.org/cidr2005/papers/P12.pdf">_Data on the Outside vs. Data on the Inside_</a>). Контракты поставщиков обычно используют некоторую форму версионирования, чтобы различать потребителей разных версий контракта.

### Контракты потребителей ###
Раз мы решили учитывать ожидания потребителей относительно схем, которые мы раскрываем при разработке нашего сервиса - и считаем, что наш поставщик знает о них - тогда нам нужно импортировать эти потребительские ожидания в поставщика. Утверждения Schematron в нашем примере очень похожи на такие тесты. Будучи использованы поставщиком, тесты могут помочь гарантировать, что поставщик продолжает выполнять свои обязательства перед своими потребителями. Внедряя эти тесты, поставщик получает лучшее представление о том, как он может развивать формат сообщений, которые он создает, не нарушая существующих функций в сообществе сервиса. И там, где предлагаемое изменение затрагивает одного или нескольких потребителей, поставщик будет иметь непосредственное представление об этой проблеме и ему будет проще обсуждать её со всеми заинтересованными сторонами, в зависимости от бизнес-факторов либо учитывая их требования, либо стимулируя доработки.

В нашем примере мы можем сказать, что набор утверждений/тестов, порожденных всеми потребителями, выражает обязательную структуру сообщений в течение некоторого периода (то есть пока тесты актуальны). Если поставщик обладал этим набором утверждений, он мог бы гарантировать, что каждое отправленное им сообщение валидно для каждого потребителя, поскольку набор утверждений является действительным и полным.

Обобщая эту структуру, мы можем отличить то, что мы уже назвали контрактом поставщика, от индивидуальных контрактных обязательств, которые возникают между поставщиком и потребителем, которые мы теперь будем называть __контрактами потребителей__. Когда поставщик принимает разумные ожидания, выраженные потребителем, он заключает контракт с потребителем.
<img src='cdc/ConsumerDrivenContracts.jpg'>
<center>_Рисунок 3: Контракты потребителей_</center>

Контракты потребителей имеют следующие характеристики:

* __Открытые и неполные__. Контракты потребителей являются открытыми и неполными в функциональности сервиса. Они выражают подмножество возможностей сервиса с точки зрения ожиданий потребителя контракта поставщика.
* __Множественные и не обязывающие__. Число контрактов потребителей пропорциональны числу потребителей сервиса, и каждый из них не является обязывающим в отношении всего набора контрактных обязательств поставщика. Не обязывающий характер отношений от потребителя к поставщику является одной из ключевых особенностей, которые отличают сервис-ориентированную архитектуру от архитектуры распределенного приложения. Потребители должны понимать, что другие потребители сервиса могут использовать его способами, совершенно отличными от их собственных. Потребители могут развиваться с разной скоростью и требовать изменений поставщика, которые потенциально могут нарушать зависимости и ожидания других потребителей. Потребитель не может предвидеть, как или когда другой потребитель будет нарушать контракт поставщика; у клиента в распределенном приложении нет таких проблем.
* __Ограниченная стабильность и неизменность__. Как и контракты поставщиков, контракты потребителей действительны в течение определенного периода времени и/или области действия.

### Контракты для потребителей ###
Контракты потребителей контракты позволяют нам размышлять о бизнес-ценности, которая существует в любой момент жизненного цикла поставщика. Выражая и утверждая ожидания контракта поставщика, контракты потребителей эффективно определяют, какие части этого контракта поставщика в настоящее время имеют ценность для бизнеса, а какие нет. Это приводит нас к предположению, что сервисные сообщества могут извлечь выгоду из того, что они сразу будут специфицироваться с точки зрения контрактов потребителей. С этой точки зрения, контракты поставщиков возникают для удовлетворения ожиданий и требований потребителей. Чтобы отразить производный характер этой новой контрактной договоренности, мы называем такие контракты поставщиков _контрактами для потребителей_ или _производными контрактами_.

Производный характер контрактов поставщиков для потребителей, добавляет гетерогенный аспект к отношениям между поставщиком сервиса и потребителем. То есть, поставщики подчиняются обязательству, которое возникает за пределами их границ. Это никоим образом не влияет на принципиально автономный характер их реализации; он просто делает явным тот факт, что сервисы успешны, только когда их используют.

Контракты для потребителей имеют следующие характеристики:
* __Закрытые и полные__. Контракт для потребителей, закрыт и завершен в отношении всего набора функций, требуемых от него его существующими потребителями. Контракт представляет собой обязательный набор элементов, необходимых для поддержки ожиданий/тестов потребителей, пока эти ожидания/тесты актуальны для приложений потребителей.
* __Единственные и не обязывающие__. Контракты поставщика являются единственным, но не обязывающим описанием функциональности сервиса, поскольку они основаны на объединении существующих ожиданий потребителей.
* __Ограниченная стабильность и неизменность__. Контракт для потребителей является стабильным и неизменным в отношении конкретного набора контрактов потребителей. Иными словами, мы можем провалидировать контракт для потребителей, лишь по определенному набору потребительских контрактов, что фактически ограничивает прямо и обратно совместимый характер контракта во времени и пространстве. Совместимость контракта остается стабильной и неизменной для определенного набора потребительских контрактов и ожиданий, но может быть изменена по мере того, как ожидания приходят и уходят.

### Сводная таблица характеристик контрактов ###
В следующей таблице приведены характеристики трех типов контракта, описанных в этой статье:
<table>
    <tr><td>__Контракт__</td><td>__Открытость__</td><td>__Полнота__</td><td>__Количество__</td><td>__Обязательность__</td><td>__Ограничение действия__</td></tr>
<tr><td>Поставщика</td><td>Закрытый</td><td>Полный</td><td>Единственный</td><td>Обязывающий</td><td>Время/область</td></tr>
<tr><td>Потребителя</td><td>Открытый</td><td>Неполный</td><td>Множественный</td><td>Не обязывающий</td><td>Время/область</td></tr>
<tr><td>Для потребителей</td><td>Закрытый</td><td>Полный</td><td>Единственный</td><td>Не обязывающий</td><td>Потребители</td></tr>      
</table>

### Реализация ###
Шаблон _Контракты для потребителей_ рекомендует создавать сообщества сервисов, используя потребительские контракты и контракты для потребителей. Однако в шаблоне не указывается, какие формы или структуры должны принимать эти контракты, а также не определяется, как ожидания/тесты потребителей сообщаются поставщику и проверяются в течение его жизненного цикла.
